In [1]:
# dependencies
import pandas as pd
import numpy as np

# name CSVs to import
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

# read CSVs into pandas
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

In [2]:
# view raw school table
school_df.head(15)

School ID            school_name      type  size   budget
0           0      Huang High School  District  2917  1910635
1           1   Figueroa High School  District  2949  1884411
2           2    Shelton High School   Charter  1761  1056600
3           3  Hernandez High School  District  4635  3022020
4           4    Griffin High School   Charter  1468   917500
5           5     Wilson High School   Charter  2283  1319574
6           6    Cabrera High School   Charter  1858  1081356
7           7     Bailey High School  District  4976  3124928
8           8     Holden High School   Charter   427   248087
9           9       Pena High School   Charter   962   585858
10         10     Wright High School   Charter  1800  1049400
11         11  Rodriguez High School  District  3999  2547363
12         12    Johnson High School  District  4761  3094650
13         13       Ford High School  District  2739  1763916
14         14     Thomas High School   Charter  1635  1043130

In [3]:
# view raw student table
student_df.head()

Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  
0             66          79  
1             94          61  
2             90          60  
3             67          58  
4             97          84

In [4]:
# merge data into single dataset
merged_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
merged_df.head()

Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  School ID      type  size   budget  
0             66          79          0  District  2917  1910635  
1             94          61          0  District  2917  1910635  
2             90          60          0  District  2917  1910635  
3             67          58          0  District  2917  1910635  
4             97          84          0  District  2917  1910635

In [5]:
merged_df.shape

(39170, 11)

## District Summary

In [6]:
# create array of unique school names
unq_school = merged_df['school_name'].unique()
# get the length of the array
school_count = len(unq_school)

# student count 
stu_count = student_df['student_name'].count()

# double check with district students
dist_students = school_df['size'].sum()

# total budget for all schools
total_budget = school_df['budget'].sum()

# average math score
avg_math = merged_df['math_score'].mean()

# average reading score
avg_reading = merged_df['reading_score'].mean()

# percent of students passing math (70 or greater)
math_num = merged_df.loc[merged_df['math_score']>= 70]['student_name'].count()
math_prcnt = math_num/stu_count*100

# percent of students passing reading (70 or greater)
reading_num = merged_df.loc[merged_df['reading_score']>= 70]['student_name'].count()
reading_prcnt = reading_num/stu_count*100

# percent of students with overall passing grades (70 or greater in both math & reading)
passing_num = merged_df.loc[(merged_df['math_score']>= 70) & (merged_df['reading_score'] >= 70)]['student_name'].count()
passing_prcnt = passing_num/stu_count*100


# display summary dataframe
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [stu_count],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math],
    "Average Reading Score": [avg_reading],
    "% Passing Math": [math_prcnt],
    "% Passing Reading": [reading_prcnt],
    "% Overall Passing": [passing_prcnt]    
})

district_summary

# map format for ease of view
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}%".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}%".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["% Overall Passing"] = district_summary["% Overall Passing"].map("{:.2f}%".format)

district_summary


Total Schools Total Students Total Budget Average Math Score  \
0             15         39,170  $24,649,428             78.99%   

  Average Reading Score % Passing Math % Passing Reading % Overall Passing  
0                81.88%         74.98%            85.81%            65.17%

In [7]:
merged_df.head()


Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  School ID      type  size   budget  
0             66          79          0  District  2917  1910635  
1             94          61          0  District  2917  1910635  
2             90          60          0  District  2917  1910635  
3             67          58          0  District  2917  1910635  
4             97          84          0  District  2917  1910635

## School Summary

In [23]:
# use groupby to sort data into fields by school name and run calcluations
by_school = merged_df.set_index('school_name').groupby(['school_name'])
sch_type = school_df.set_index('school_name')['type']
stu_total = by_school['Student ID'].count()
sch_budget = school_df.set_index('school_name')['budget']
stu_budget = sch_budget/stu_total
sch_avg_math = by_school['math_score'].mean()
sch_avg_rdg = by_school['reading_score'].mean()
sch_math_prcnt = merged_df.loc[merged_df['math_score']>= 70].groupby('school_name')['Student ID'].count()/stu_total*100
sch_reading_prcnt = merged_df.loc[merged_df['reading_score']>= 70].groupby('school_name')['Student ID'].count()/stu_total*100
overall_prcnt = merged_df.loc[(merged_df['math_score']>= 70) & (merged_df['reading_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_total*100

# create dataframe to hold results
school_summary = pd.DataFrame({
    "School Type": sch_type,
    "Total Students": stu_total,
    "Total School Budget": sch_budget,
    "Per Student Budget": stu_budget,
    "Average Math Score": sch_avg_math,
    "Average Reading Score": sch_avg_rdg,
    "% Passing Math": sch_math_prcnt,
    "% Passing Reading": sch_reading_prcnt,
    "% Overall Passing": overall_prcnt
})

# map format for readability
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:.0f}".format)
school_summary["Average Math Score"] = school_summary["Average Math Score"].map("{:.2f}%".format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map("{:.2f}%".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}%".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}%".format)
school_summary["% Overall Passing"] = school_summary["% Overall Passing"].map("{:.2f}%".format)

school_summary


School Type Total Students Total School Budget  \
Bailey High School       District          4,976          $3,124,928   
Cabrera High School       Charter          1,858          $1,081,356   
Figueroa High School     District          2,949          $1,884,411   
Ford High School         District          2,739          $1,763,916   
Griffin High School       Charter          1,468            $917,500   
Hernandez High School    District          4,635          $3,022,020   
Holden High School        Charter            427            $248,087   
Huang High School        District          2,917          $1,910,635   
Johnson High School      District          4,761          $3,094,650   
Pena High School          Charter            962            $585,858   
Rodriguez High School    District          3,999          $2,547,363   
Shelton High School       Charter          1,761          $1,056,600   
Thomas High School        Charter          1,635          $1,043,130   
Wilson High School        Charter          2,283          $1,319,574   
Wright High School        Charter          1,800          $1,049,400   

                      Per Student Budget Average Math Score  \
Bailey High School                  $628             77.05%   
Cabrera High School                 $582             83.06%   
Figueroa High School                $639             76.71%   
Ford High School                    $644             77.10%   
Griffin High School                 $625             83.35%   
Hernandez High School               $652             77.29%   
Holden High School                  $581             83.80%   
Huang High School                   $655             76.63%   
Johnson High School                 $650             77.07%   
Pena High School                    $609             83.84%   
Rodriguez High School               $637             76.84%   
Shelton High School                 $600             83.36%   
Thomas High School                  $638             83.42%   
Wilson High School                  $578             83.27%   
Wright High School                  $583             83.68%   

                      Average Reading Score % Passing Math % Passing Reading  \
Bailey High School                   81.03%         66.68%            81.93%   
Cabrera High School                  83.98%         94.13%            97.04%   
Figueroa High School                 81.16%         65.99%            80.74%   
Ford High School                     80.75%         68.31%            79.30%   
Griffin High School                  83.82%         93.39%            97.14%   
Hernandez High School                80.93%         66.75%            80.86%   
Holden High School                   83.81%         92.51%            96.25%   
Huang High School                    81.18%         65.68%            81.32%   
Johnson High School                  80.97%         66.06%            81.22%   
Pena High School                     84.04%         94.59%            95.95%   
Rodriguez High School                80.74%         66.37%            80.22%   
Shelton High School                  83.73%         93.87%            95.85%   
Thomas High School                   83.85%         93.27%            97.31%   
Wilson High School                   83.99%         93.87%            96.54%   
Wright High School                   83.95%         93.33%            96.61%   

                      % Overall Passing  
Bailey High School               54.64%  
Cabrera High School              91.33%  
Figueroa High School             53.20%  
Ford High School                 54.29%  
Griffin High School              90.60%  
Hernandez High School            53.53%  
Holden High School               89.23%  
Huang High School                53.51%  
Johnson High School              53.54%  
Pena High School                 90.54%  
Rodriguez High School            52.99%  
Shelton High School              89.89%  
Thomas High School               

## Top Performing Schools (By % Overall Passing)

In [9]:
top_5 = school_summary.sort_values('% Overall Passing', ascending = False)
top_5.head(5)

School Type Total Students Total School Budget  \
Cabrera High School     Charter          1,858          $1,081,356   
Thomas High School      Charter          1,635          $1,043,130   
Griffin High School     Charter          1,468            $917,500   
Wilson High School      Charter          2,283          $1,319,574   
Pena High School        Charter            962            $585,858   

                    Per Student Budget Average Math Score  \
Cabrera High School               $582             83.06%   
Thomas High School                $638             83.42%   
Griffin High School               $625             83.35%   
Wilson High School                $578             83.27%   
Pena High School                  $609             83.84%   

                    Average Reading Score % Passing Math % Passing Reading  \
Cabrera High School                83.98%         94.13%            97.04%   
Thomas High School                 83.85%         93.27%            97.31%   
Griffin High School                83.82%         93.39%            97.14%   
Wilson High School                 83.99%         93.87%            96.54%   
Pena High School                   84.04%         94.59%            95.95%   

                    % Overall Passing  
Cabrera High School            91.33%  
Thomas High School             90.95%  
Griffin High School            90.60%  
Wilson High School             90.58%  
Pena High School               90.54%

## Bottom Performing Schools (By % Overall Passing)

In [10]:
bottom_5 = school_summary.sort_values('% Overall Passing', ascending = True)
bottom_5.head(5)

School Type Total Students Total School Budget  \
Rodriguez High School    District          3,999          $2,547,363   
Figueroa High School     District          2,949          $1,884,411   
Huang High School        District          2,917          $1,910,635   
Hernandez High School    District          4,635          $3,022,020   
Johnson High School      District          4,761          $3,094,650   

                      Per Student Budget Average Math Score  \
Rodriguez High School               $637             76.84%   
Figueroa High School                $639             76.71%   
Huang High School                   $655             76.63%   
Hernandez High School               $652             77.29%   
Johnson High School                 $650             77.07%   

                      Average Reading Score % Passing Math % Passing Reading  \
Rodriguez High School                80.74%         66.37%            80.22%   
Figueroa High School                 81.16%         65.99%            80.74%   
Huang High School                    81.18%         65.68%            81.32%   
Hernandez High School                80.93%         66.75%            80.86%   
Johnson High School                  80.97%         66.06%            81.22%   

                      % Overall Passing  
Rodriguez High School            52.99%  
Figueroa High School             53.20%  
Huang High School                53.51%  
Hernandez High School            53.53%  
Johnson High School              53.54%

## Math Scores by Grade

In [11]:
# create values for math score by grade

ninth_math = merged_df.loc[merged_df['grade'] == '9th'].groupby('school_name')['math_score'].mean()
tenth_math = merged_df.loc[merged_df['grade'] == '10th'].groupby('school_name')['math_score'].mean()
eleventh_math = merged_df.loc[merged_df['grade'] == '11th'].groupby('school_name')['math_score'].mean()
twelfth_math = merged_df.loc[merged_df['grade'] == '12th'].groupby('school_name')['math_score'].mean()

math_scores_by_grade = pd.DataFrame({
    "9th": ninth_math,
    "10th": tenth_math,
    "11th": eleventh_math,
    "12th": twelfth_math
})

math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.2f}%".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.2f}%".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.2f}%".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.2f}%".format)

math_scores_by_grade.index.name = "School"

math_scores_by_grade

9th    10th    11th    12th
School                                               
Bailey High School     77.08%  77.00%  77.52%  76.49%
Cabrera High School    83.09%  83.15%  82.77%  83.28%
Figueroa High School   76.40%  76.54%  76.88%  77.15%
Ford High School       77.36%  77.67%  76.92%  76.18%
Griffin High School    82.04%  84.23%  83.84%  83.36%
Hernandez High School  77.44%  77.34%  77.14%  77.19%
Holden High School     83.79%  83.43%  85.00%  82.86%
Huang High School      77.03%  75.91%  76.45%  77.23%
Johnson High School    77.19%  76.69%  77.49%  76.86%
Pena High School       83.63%  83.37%  84.33%  84.12%
Rodriguez High School  76.86%  76.61%  76.40%  77.69%
Shelton High School    83.42%  82.92%  83.38%  83.78%
Thomas High School     83.59%  83.09%  83.50%  83.50%
Wilson High School     83.09%  83.72%  83.20%  83.04%
Wright High School     83.26%  84.01%  83.84%  83.64%

## Reading Scores by Grade

In [12]:
# create values for reading score by grade

ninth_read = merged_df.loc[merged_df['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
tenth_read = merged_df.loc[merged_df['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
eleventh_read = merged_df.loc[merged_df['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
twelfth_read = merged_df.loc[merged_df['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_read,
    "10th": tenth_read,
    "11th": eleventh_read,
    "12th": twelfth_read
})

reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:.2f}%".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:.2f}%".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:.2f}%".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:.2f}%".format)

reading_scores_by_grade.index.name = "School"

reading_scores_by_grade

9th    10th    11th    12th
School                                               
Bailey High School     81.30%  80.91%  80.95%  80.91%
Cabrera High School    83.68%  84.25%  83.79%  84.29%
Figueroa High School   81.20%  81.41%  80.64%  81.38%
Ford High School       80.63%  81.26%  80.40%  80.66%
Griffin High School    83.37%  83.71%  84.29%  84.01%
Hernandez High School  80.87%  80.66%  81.40%  80.86%
Holden High School     83.68%  83.32%  83.82%  84.70%
Huang High School      81.29%  81.51%  81.42%  80.31%
Johnson High School    81.26%  80.77%  80.62%  81.23%
Pena High School       83.81%  83.61%  84.34%  84.59%
Rodriguez High School  80.99%  80.63%  80.86%  80.38%
Shelton High School    84.12%  83.44%  84.37%  82.78%
Thomas High School     83.73%  84.25%  83.59%  83.83%
Wilson High School     83.94%  84.02%  83.76%  84.32%
Wright High School     83.83%  83.81%  84.16%  84.07%

## Scores by School Spending

In [15]:
# use groupby to sort data into fields by school name and run calcluations
by_school = merged_df.set_index('school_name').groupby(['school_name'])
sch_type = school_df.set_index('school_name')['type']
stu_total = by_school['Student ID'].count()
sch_budget = school_df.set_index('school_name')['budget']
stu_budget = sch_budget/stu_total
sch_avg_math = by_school['math_score'].mean()
sch_avg_rdg = by_school['reading_score'].mean()
sch_math_prcnt = merged_df.loc[merged_df['math_score']>= 70].groupby('school_name')['Student ID'].count()/stu_total*100
sch_reading_prcnt = merged_df.loc[merged_df['reading_score']>= 70].groupby('school_name')['Student ID'].count()/stu_total*100
overall_prcnt = merged_df.loc[(merged_df['math_score']>= 70) & (merged_df['reading_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_total*100

# create dataframe to hold results
school_stats = pd.DataFrame({
    "School Type": sch_type,
    "Total Students": stu_total,
    "Total School Budget": sch_budget,
    "Per Student Budget": stu_budget,
    "Average Math Score": sch_avg_math,
    "Average Reading Score": sch_avg_rdg,
    "% Passing Math": sch_math_prcnt,
    "% Passing Reading": sch_reading_prcnt,
    "% Overall Passing": overall_prcnt
})

school_stats

School Type  Total Students  Total School Budget  \
Bailey High School       District            4976              3124928   
Cabrera High School       Charter            1858              1081356   
Figueroa High School     District            2949              1884411   
Ford High School         District            2739              1763916   
Griffin High School       Charter            1468               917500   
Hernandez High School    District            4635              3022020   
Holden High School        Charter             427               248087   
Huang High School        District            2917              1910635   
Johnson High School      District            4761              3094650   
Pena High School          Charter             962               585858   
Rodriguez High School    District            3999              2547363   
Shelton High School       Charter            1761              1056600   
Thomas High School        Charter            1635              1043130   
Wilson High School        Charter            2283              1319574   
Wright High School        Charter            1800              1049400   

                       Per Student Budget  Average Math Score  \
Bailey High School                  628.0           77.048432   
Cabrera High School                 582.0           83.061895   
Figueroa High School                639.0           76.711767   
Ford High School                    644.0           77.102592   
Griffin High School                 625.0           83.351499   
Hernandez High School               652.0           77.289752   
Holden High School                  581.0           83.803279   
Huang High School                   655.0           76.629414   
Johnson High School                 650.0           77.072464   
Pena High School                    609.0           83.839917   
Rodriguez High School               637.0           76.842711   
Shelton High School                 600.0           83.359455   
Thomas High School                  638.0           83.418349   
Wilson High School                  578.0           83.274201   
Wright High School                  583.0           83.682222   

                       Average Reading Score  % Passing Math  \
Bailey High School                 81.033963       66.680064   
Cabrera High School                83.975780       94.133477   
Figueroa High School               81.158020       65.988471   
Ford High School                   80.746258       68.309602   
Griffin High School                83.816757       93.392371   
Hernandez High School              80.934412       66.752967   
Holden High School                 83.814988       92.505855   
Huang High School                  81.182722       65.683922   
Johnson High School                80.966394       66.057551   
Pena High School                   84.044699       94.594595   
Rodriguez High School              80.744686       66.366592   
Shelton High School                83.725724       93.867121   
Thomas High School                 83.848930       93.272171   
Wilson High School                 83.989488       93.867718   
Wright High School                 83.955000       93.333333   

                       % Passing Reading  % Overall Passing  
Bailey High School             81.933280          54.642283  
Cabrera High School            97.039828          91.334769  
Figueroa High School           80.739234          53.204476  
Ford High School               79.299014          54.289887  
Griffin High School            97.138965          90.599455  
Hernandez High School          80.862999          53.527508  
Holden High School             96.252927          89.227166  
Huang High School              81.316421          53.513884  
Johnson High School            81.222432          53.539172  
Pena High School               95.945946          90.540541  
Rodriguez High School          80.220055          52.988247  
Shelton High School        

In [16]:
school_stats.dtypes

School Type               object
Total Students             int64
Total School Budget        int64
Per Student Budget       float64
Average Math Score       float64
Average Reading Score    float64
% Passing Math           float64
% Passing Reading        float64
% Overall Passing        float64
dtype: object

In [21]:
bins = [0, 584, 629, 644, 680]
budget_labels = ['<$585', '$585-630', '$630-645', '$645-680']

pd.cut(school_stats["Per Student Budget"], bins, labels=budget_labels)

Bailey High School       $585-630
Cabrera High School         <$585
Figueroa High School     $630-645
Ford High School         $630-645
Griffin High School      $585-630
Hernandez High School    $645-680
Holden High School          <$585
Huang High School        $645-680
Johnson High School      $645-680
Pena High School         $585-630
Rodriguez High School    $630-645
Shelton High School      $585-630
Thomas High School       $630-645
Wilson High School          <$585
Wright High School          <$585
Name: Per Student Budget, dtype: category
Categories (4, object): ['<$585' < '$585-630' < '$630-645' < '$645-680']

In [19]:
school_stats["Spending Ranges (Per Student)"] = pd.cut(school_stats["Per Student Budget"], bins, labels=budget_labels)
school_stats

School Type  Total Students  Total School Budget  \
Bailey High School       District            4976              3124928   
Cabrera High School       Charter            1858              1081356   
Figueroa High School     District            2949              1884411   
Ford High School         District            2739              1763916   
Griffin High School       Charter            1468               917500   
Hernandez High School    District            4635              3022020   
Holden High School        Charter             427               248087   
Huang High School        District            2917              1910635   
Johnson High School      District            4761              3094650   
Pena High School          Charter             962               585858   
Rodriguez High School    District            3999              2547363   
Shelton High School       Charter            1761              1056600   
Thomas High School        Charter            1635              1043130   
Wilson High School        Charter            2283              1319574   
Wright High School        Charter            1800              1049400   

                       Per Student Budget  Average Math Score  \
Bailey High School                  628.0           77.048432   
Cabrera High School                 582.0           83.061895   
Figueroa High School                639.0           76.711767   
Ford High School                    644.0           77.102592   
Griffin High School                 625.0           83.351499   
Hernandez High School               652.0           77.289752   
Holden High School                  581.0           83.803279   
Huang High School                   655.0           76.629414   
Johnson High School                 650.0           77.072464   
Pena High School                    609.0           83.839917   
Rodriguez High School               637.0           76.842711   
Shelton High School                 600.0           83.359455   
Thomas High School                  638.0           83.418349   
Wilson High School                  578.0           83.274201   
Wright High School                  583.0           83.682222   

                       Average Reading Score  % Passing Math  \
Bailey High School                 81.033963       66.680064   
Cabrera High School                83.975780       94.133477   
Figueroa High School               81.158020       65.988471   
Ford High School                   80.746258       68.309602   
Griffin High School                83.816757       93.392371   
Hernandez High School              80.934412       66.752967   
Holden High School                 83.814988       92.505855   
Huang High School                  81.182722       65.683922   
Johnson High School                80.966394       66.057551   
Pena High School                   84.044699       94.594595   
Rodriguez High School              80.744686       66.366592   
Shelton High School                83.725724       93.867121   
Thomas High School                 83.848930       93.272171   
Wilson High School                 83.989488       93.867718   
Wright High School                 83.955000       93.333333   

                       % Passing Reading  % Overall Passing  \
Bailey High School             81.933280          54.642283   
Cabrera High School            97.039828          91.334769   
Figueroa High School           80.739234          53.204476   
Ford High School               79.299014          54.289887   
Griffin High School            97.138965          90.599455   
Hernandez High School          80.862999          53.527508   
Holden High School             96.252927          89.227166   
Huang High School              81.316421          53.513884   
Johnson High School            81.222432          53.539172   
Pena High School               95.945946          90.540541   
Rodriguez High School          80.220055          52.988247   
Shelton High Sc

In [26]:
spend_group = school_stats.groupby("Spending Ranges (Per Student)")

spend_group[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]].mean()


Average Math Score  Average Reading Score  \
Spending Ranges (Per Student)                                              
<$585                                   83.455399              83.933814   
$585-630                                81.899826              83.155286   
$630-645                                78.518855              81.624473   
$645-680                                76.997210              81.027843   

                               % Passing Math  % Passing Reading  \
Spending Ranges (Per Student)                                      
<$585                               93.460096          96.610877   
$585-630                            87.133538          92.718205   
$630-645                            73.484209          84.391793   
$645-680                            66.164813          81.133951   

                               % Overall Passing  
Spending Ranges (Per Student)                     
<$585                                  90.369459  
$585-630                               81.418596  
$630-645                               62.857656  
$645-680                               53.526855